# Binance Test Net API Research

In [7]:
import pandas as pd
import numpy as np
from binance.client import Client
import datetime

import requests
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import websocket
import json

import plotly.graph_objs as go

In [ ]:
### Testnet URLs
# https://testnet.binance.vision/api
# wss://testnet.binance.vision/ws-api/v3
# wss://testnet.binance.vision/ws
# wss://testnet.binance.vision/stream

# Binance Spot Test Network: https://testnet.binance.vision/
# API Key: odwF9bVsSsxjZnckgbSu3NfUgGqqJ2sow4OelwjEttIBB08r3Z5umQL0A03lp2Gd
# Secret key: Q3bcPKvbvlVpzv5BQe3lj7EkWdRhevEp24Oi7TENce6xO0FiXUNQKDa47QTyyKcK

In [24]:
# Global variables for storing price data
timestamps = []
price = {'open':[], 'high':[], 'low':[], 'close':[]}
max_messages = 5  # Maximum number of messages to receive before closing websocket

# Initial dashboard graph
fig = go.Figure()

def time_conv(timestamp):
    return datetime.datetime.fromtimestamp(timestamp/ 1000).strftime('%Y-%m-%d %H:%M:%S.%f')[:-4]

def on_error(ws, error):
    print("WebSocket error:", error)

def on_open(ws):
    print("WebSocket connection opened")

def on_close(ws):
    fig.show()
    print('done')
    print("WebSocket connection closed")

def on_message(ws, message):
    global data
    data = json.loads(message)
    # Extract timestamp and price from the data
    timestamp = data['E']

    # Append timestamp and price to the lists
    timestamps.append(time_conv(data['E']))
    price['open'].append(data['o'])
    price['high'].append(data['h'])
    price['low'].append(data['l'])
    price['close'].append(data['c'])

    # Update dashboard graph with new data
    update_graph()

    # Terminate the WebSocket connection after reaching the maximum number of messages
    if len(timestamps) >= max_messages:
        ws.close()
    
'''# Define the callback to update the graph
@app.callback(Output('live-graph', 'figure'),
              events=[Event('interval', 'interval')])'''
def update_graph():
    # Update the graph
    fig.update_xaxes(range=[min(timestamps), max(timestamps)])
    fig.update_yaxes(range=[min(price['open']), max(price['open'])])
    fig.update_traces(
        x=timestamps,
        open=price['open'],
        high=price['high'],
        low=price['low'],
        close=price['close']
    )
    #return fig

# Binance WebSocket API endpoint for BTCUSDT symbol
ws_url = 'wss://testnet.binance.vision/ws/btcusdt@miniTicker'

# Start WebSocket connection
ws = websocket.WebSocketApp(
        url=ws_url,
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close
    )
ws.run_forever()

WebSocket connection opened


False

In [29]:
ws.keep_running

False

In [10]:
ws.close()

In [25]:
data

{'e': '24hrMiniTicker',
 'E': 1685569253923,
 's': 'BTCUSDT',
 'c': '27083.51000000',
 'o': '27791.84000000',
 'h': '27831.69000000',
 'l': '24050.00000000',
 'v': '4067.73027800',
 'q': '110648845.41496455'}

In [11]:
timestamps

['2023-05-31 14:29:29.27',
 '2023-05-31 14:29:29.27',
 '2023-05-31 14:29:36.84',
 '2023-05-31 14:29:37.83',
 '2023-05-31 14:29:38.83']

In [12]:
price

{'open': ['27819.79000000',
  '27819.79000000',
  '27819.79000000',
  '27819.79000000',
  '27819.79000000'],
 'high': ['27831.69000000',
  '27831.69000000',
  '27831.69000000',
  '27831.69000000',
  '27831.69000000'],
 'low': ['24050.00000000',
  '24050.00000000',
  '24050.00000000',
  '24050.00000000',
  '24050.00000000'],
 'close': ['27049.26000000',
  '27049.26000000',
  '27049.26000000',
  '27049.59000000',
  '27050.01000000']}

In [28]:

from dash import dash, html, dcc, Input, Output

# custom files
import backend

app = dash.Dash(__name__)

# Google Analytics tag
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        <!-- Google tag (gtag.js) -->
        <script async src="https://www.googletagmanager.com/gtag/js?id=G-WMWKTVG0WM"></script>
        <script>
        window.dataLayer = window.dataLayer || [];
        function gtag(){dataLayer.push(arguments);}
        gtag('js', new Date());

        gtag('config', 'G-WMWKTVG0WM');
        </script>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# retrieving components for dashboard display
crypto_graph = make_graph(app)
updated = last_updated()
time_interval=1000

# dashboard layout
app.layout = html.Div(children=[
    html.H2(children=f"Last update: {updated} UTC"),
    #html.Div(children="Menu"),
    dcc.Dropdown(['Coin Prices (Real Time)', 'Our Trades', 'Our Returns'], 'Current Coin Prices'),
    dcc.Graph(id='crypto-graph', figure=crypto_graph),
    dcc.Interval(interval=time_interval)
    #, generate_table(data)

])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

NameError: name 'response' is not defined

In [22]:
print(fig)

Figure({
    'data': [],
    'layout': {'template': '...',
               'xaxis': {'range': ['2023-05-31 14:30:53.83', '2023-05-31 14:31:03.26']},
               'yaxis': {'range': ['27819.78000000', '27819.79000000']}}
})


In [23]:
price

{'open': ['27819.79000000',
  '27819.79000000',
  '27819.78000000',
  '27819.78000000',
  '27819.79000000'],
 'high': ['27831.69000000',
  '27831.69000000',
  '27831.69000000',
  '27831.69000000',
  '27831.69000000'],
 'low': ['24050.00000000',
  '24050.00000000',
  '24050.00000000',
  '24050.00000000',
  '24050.00000000'],
 'close': ['27062.12000000',
  '27062.14000000',
  '27062.18000000',
  '27062.18000000',
  '27062.21000000']}